In [74]:
import pandas as pd
import numpy as np

In [75]:
ticket_flights = pd.read_csv('3-hive/ticket_flights/ticket_flights.csv', names = ['ticket_no', 'flight_id', 'fare_conditions', 'amount']
                   , sep = ',', header=None)

flights = pd.read_csv('3-hive/flights/flights.csv'
                      , names = ['flight_id', 'flight_no', 'scheduled_departure', 'scheduled_arrival'
                                , 'departure_airport', 'arrival_airport', 'status', 'aircraft_code'
                                , 'actual_departure', 'actual_arrival'
                                ]
                   , sep = ',', header=None)

flights['scheduled_departure'] = pd.to_datetime(flights['scheduled_departure'])
flights['scheduled_arrival'] = pd.to_datetime(flights['scheduled_arrival'])
flights['actual_departure'] = pd.to_datetime(flights['actual_departure'])
flights['actual_arrival'] = pd.to_datetime(flights['actual_arrival'])

In [77]:
merged_data = pd.merge(ticket_flights, 
                  flights,
                  left_on='flight_id',
                  right_on='flight_id',
                  how='inner')

In [192]:
merged_data_cut = merged_data[merged_data['status'].isin(['Arrived'])]

In [193]:
merged_data_cut['delay'] = (merged_data_cut['actual_departure'] - merged_data_cut['scheduled_departure']).dt.seconds/60

<ipython-input-193-22dc4d0ac39a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_cut['delay'] = (merged_data_cut['actual_departure'] - merged_data_cut['scheduled_departure']).dt.seconds/60


In [194]:
merged_data_cut['delay']= merged_data_cut['delay'].fillna(0)

<ipython-input-194-34eb794155cc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_cut['delay']= merged_data_cut['delay'].fillna(0)


In [195]:
agg_data = merged_data_cut.groupby(['flight_no', 'fare_conditions']).agg(
      min_price = ('amount', np.min)
    , cnt_seats = ('fare_conditions', 'count')
    , sum_delay = ('delay', np.sum)
).reset_index()

In [196]:
for f in agg_data['flight_no'].unique():
    bp = agg_data[(agg_data['flight_no'] == f) & (agg_data['fare_conditions'] == 'Business')]['min_price'].values
    ep = agg_data[(agg_data['flight_no'] == f) & (agg_data['fare_conditions'] == 'Economy')]['min_price'].values
    bs = agg_data[(agg_data['flight_no'] == f) & (agg_data['fare_conditions'] == 'Business')]['cnt_seats'].values
    bp = bp[0] if bp.size > 0 else 0
    ep = ep[0] if ep.size > 0 else 0
    bs = bs[0] if bs.size > 0 else 0
    cnt = sum(agg_data[agg_data['flight_no'] == f]['cnt_seats'].values)
    sss = sum(agg_data[agg_data['flight_no'] == f]['sum_delay'].values)
    metric = (0 if sss == 0 else 1/(sss/cnt)) + 0.003*(bp + ep) + 0.07*bs
    print(f, metric)

PG0012 36.93713474025974
PG0013 206.61453195063726
PG0014 58.752659995345596
PG0015 56.41274131274131
PG0016 56.656701030927834
PG0019 28.613102526995437
PG0020 28.864675324675325
PG0029 16.022199592668024
PG0030 15.937947997189038
PG0032 16.04070796460177
PG0035 26.44170854271357
PG0038 26.48983050847458
PG0039 55.089101294744864
PG0040 60.041970868812065
PG0041 62.74780244676031
PG0042 71.09963235294117
PG0043 29.803896103896104
PG0044 29.886531007751937
PG0045 43.88
PG0046 44.139361702127665
PG0049 36.950348953140576
PG0050 21.71073382380683
PG0051 21.69267548321465
PG0052 57.964624348900855
PG0053 58.1202216066482
PG0055 11.154450971531856
PG0056 11.430250990752972
PG0059 21.39673418768086
PG0061 21.404693140794226
PG0062 16.523863636363636
PG0063 16.32038949542638
PG0064 535.3934224598929
PG0065 535.2817721518987
PG0066 37.80585424133811
PG0067 37.68541748333987
PG0070 14.928318584070798
PG0071 14.802994886778672
PG0072 354.69264049955393
PG0073 421.9426732673267
PG0077 13.3049180

PG0638 108.35266684920255
PG0639 65.20661553211889
PG0640 65.24642609299097
PG0641 20.748259808096293
PG0642 20.804887806575056
PG0643 52.21178707224335
PG0644 52.34736842105263
PG0645 21.05781198820832
PG0646 21.286524204099432
PG0647 29.541414141414144
PG0648 29.473503521126762
PG0649 57.7897093328664
PG0650 52.01538280077204
PG0651 71.72272727272728
PG0652 71.77643207855975
PG0655 46.85415419695027
PG0656 47.09813976872801
PG0657 47.37342306677224
PG0658 47.28198454656364
PG0659 68.55195369030392
PG0660 68.7208395802099
PG0661 67.25692307692309
PG0662 67.33125000000001
PG0663 365.8091347588265
PG0664 364.94531049250537
PG0667 52.247064096817574
PG0668 52.55148514851486
PG0669 52.84413472706156
PG0670 52.85417244247297
PG0671 68.1824419778002
PG0672 67.85885053250091
PG0673 50.48209394168557
PG0674 50.458968507530805
PG0675 13.251259978087338
PG0676 13.244109436069236
PG0677 218.8642648706092
PG0678 215.05351902511813
PG0679 45.281073721065944
PG0680 45.337299035369774
PG0683 93.3509